# Notebook 15: Rayleigh-Bénard Convection (with physical units)

<div style="float: right; width: 40%">
    
![](media/AnnulusConvectionModel.png)

</div>


We'll look at a convection problem which couples Stokes Flow with time-dependent advection/diffusion to give simple Rayleigh-Bénard convection model. 

$$
-\nabla \cdot
    \left[
            \frac{\eta}{2}\left( \nabla \mathbf{u} + \nabla \mathbf{u}^T \right) -  p \mathbf{I} \right] =
     -\rho_0 \alpha T \mathbf{g} 
$$
$$
\nabla \cdot \mathbf{u} = 0
$$

$\eta$ is viscosity, $p$ is pressure, $\rho_0$ is a reference density, $\alpha$ is thermal expansivity, and $T$ is the temperature. Here we explicitly express density variations in terms of temperature variations.

Thermal evolution is given by
$$
\frac{\partial T}{\partial t} - \mathbf{u}\cdot\nabla T = \kappa \nabla^2 T 
$$
where the velocity, $\mathbf{u}$ is the result of the Stokes flow calculation. $\kappa$ is the thermal diffusivity (compare this with Notebook 4).  

The starting point is our previous notebook where we solved for Stokes
flow in a cylindrical annulus geometry. We then add an advection-diffusion 
solver to evolve temperature. The Stokes buoyancy force is proportional to the
temperature anomaly, and the velocity solution is fed back into the 
temperature advection term. The timestepping loop is written by
hand because usually you will want to do some analysis or output some checkpoints.

To read more about the applications of simple mantle convection models like this one, see (for example) Schubert et al, 2001.



In [1]:
#|  echo: false  # Hide in html version

# This is required to fix pyvista
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import numpy as np
import sympy
import underworld3 as uw

In [3]:
uw.__file__

'/Users/lmoresi/+Underworld/underworld3-pixi/.pixi/envs/amr-dev/lib/python3.12/site-packages/underworld3/__init__.py'

In [4]:
# Step 1: Create Model FIRST
model = uw.Model()

# Step 2: Define the unit system

model.set_reference_quantities(
    length=uw.quantity(1000, "km"),
    diffusivity=uw.quantity(1e-6, "m**2/s"),
    viscosity=uw.quantity(1e21, "Pa.s"),
    temperature=uw.quantity(1000, "K"), 
    verbose=False,
    nondimensional_scaling=True, # The default !
)

uw.use_nondimensional_scaling(True)

outer_radius = uw.expression(r"r_o", uw.quantity(6370, "km"), "outer radius")
inner_radius = uw.expression(r"r_i", outer_radius * 0.55, "inner radius")
mantle_thickness = uw.expression(r"d_m", outer_radius.sym - inner_radius.sym, "mantle thickness")

velocity_phys = uw.quantity(1, "cm/year")    # Horizontal velocity
T_outer = uw.quantity(273, "K")               # Left boundary temperature (cold)
T_inner = uw.quantity(283, "K")             # Right boundary temperature (hot)

alpha = uw.expression(r"\alpha", uw.quantity(1e-5, "1/K"), "thermal expansivity")
kappa = uw.expression(r"\kappa", uw.quantity(1e-6, "m**2/s"), "thermal diffusivity")    
rho = uw.expression(r"\rho", uw.quantity(3000, "kg/m**3"), "density")    
eta_0 = uw.expression(r"\eta_0", uw.quantity(7e21, "Pa.s"), "mantle viscosity")    
gravity = uw.expression(r"g", uw.quantity(10, "m/s**2"), "gravitational acceleration")
deltaT = uw.expression(r"\Delta T", T_inner - T_outer, "temperature drop")

✓ Units system active with automatic non-dimensionalization


In [5]:
rayleigh_number = gravity * rho * alpha * deltaT * mantle_thickness**3 / (kappa * eta_0)

In [6]:
rayleigh_number

d_m**3*\Delta T*\alpha*\rho*g/(\kappa*\eta_0)

In [7]:
mantle_thickness = outer_radius - inner_radius

res = 5

meshball = uw.meshing.Annulus(
    radiusOuter=outer_radius,
    radiusInner=inner_radius,
    cellSize= mantle_thickness / res,
    qdegree=3,
)

# Coordinate directions etc
x, y = meshball.CoordinateSystem.X
r, th = meshball.CoordinateSystem.R
unit_rvec = meshball.CoordinateSystem.unit_e_0

# Orientation of surface normals
Gamma_N = meshball.Gamma / sympy.sqrt(meshball.Gamma.dot(meshball.Gamma))
Gamma_N = unit_rvec

In [8]:
Gamma_N

Matrix([[N.x/sqrt(N.x**2 + N.y**2), N.y/sqrt(N.x**2 + N.y**2)]])

In [9]:
uw.function.evaluate(rayleigh_number, meshball.X.coords[0]).squeeze()

array(10094.36613412)

In [10]:
# Mesh variables for the unknowns

v_soln = uw.discretisation.MeshVariable("V0", meshball, 2, degree=2, varsymbol=r"{v_0}", units="cm/yr")
p_soln = uw.discretisation.MeshVariable("p", meshball, 1, degree=1, continuous=False, units="MPa")
t_soln = uw.discretisation.MeshVariable("T", meshball, 1, degree=3, continuous=True, units="K")
t_0    = uw.discretisation.MeshVariable("T0", meshball, 1, degree=3, continuous=True, units="K")

### Create linked solvers

We create the Stokes solver as we did in the previous notebook. 
The buoyancy force is proportional to the temperature anomaly
(`t_soln`). Solvers can either be provided with unknowns as pre-defined
meshVariables, or they will define their own. When solvers are coupled,
explicitly defining unknowns makes everything clearer.

The advection-diffusion solver evolved `t_soln` using the Stokes
velocity `v_soln` in the fluid-transport term. 

### Curved, free-slip boundaries

In the annulus, a free slip boundary corresponds to zero radial 
velocity. However, in this mesh, $v_r$ is not one of the unknowns
($\mathbf{v} = (v_x, v_y)$). We apply a non linear boundary condition that
penalises $v_r$ on the boundary as discussed previously in Example 5. 

In [11]:
stokes = uw.systems.Stokes(
    meshball,
    velocityField=v_soln,
    pressureField=p_soln,
)

stokes.bodyforce = gravity * rho * alpha * t_soln * unit_rvec

stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes.constitutive_model.Parameters.shear_viscosity_0 = eta_0
stokes.tolerance = 1.0e-3

# stokes.petsc_options.setValue("ksp_monitor", None)
# stokes.petsc_options.setValue("snes_monitor", None)
stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"

penalty = 1000000 * uw.non_dimensionalise(eta_0)

stokes.add_natural_bc(penalty * Gamma_N.dot(v_soln) * Gamma_N, "Upper")
stokes.add_natural_bc(penalty * Gamma_N.dot(v_soln) * Gamma_N, "Lower")
# stokes.add_dirichlet_bc((uw.quantity(0,"mm/Myr"),uw.quantity(0,"mm/Myr")), "Lower")

In [12]:
# Create solver for the energy equation (Advection-Diffusion of temperature)

adv_diff = uw.systems.AdvDiffusion(
    meshball,
    u_Field=t_soln,
    V_fn=v_soln,
    order=2,
    verbose=False,
)

adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = kappa

## Boundary conditions for this solver

adv_diff.add_dirichlet_bc(T_inner, "Lower")
adv_diff.add_dirichlet_bc(T_outer, "Upper")

adv_diff.petsc_options.setValue("snes_rtol", 0.001)
adv_diff.petsc_options.setValue("ksp_rtol", 0.0001)
# adv_diff.petsc_options.setValue("snes_monitor", None)
# adv_diff.petsc_options.setValue("ksp_monitor", None)

### Initial condition

We need to set an initial condition for the temperature field as the 
coupled system is an initial value problem. Choose whatever works but
remember that the boundary conditions will over-rule values you set on 
the lower and upper boundaries.

In [13]:
# Initial temperature

r_prime = uw.expression(r"r'", (r - inner_radius) / (mantle_thickness), "R prime")

init_t = T_outer + deltaT * (1 - r_prime) + 0.1 * deltaT * (sympy.sin(3 * th) * sympy.cos(np.pi * r_prime)) 

t_soln.array[...] = uw.function.evaluate(init_t, t_soln.coords)
t_0.data[...] = t_soln.data[...]

#### Initial velocity solve

The first solve allows us to determine the magnitude of the velocity field 
and is useful to keep separated to check convergence rates etc. 

For non-linear problems, we usually need an initial guess using a 
reasonably close linear problem. 

`zero_init_guess` is used to reset any information in the vector of 
unknowns (i.e. do not use any initial information if `zero_init_guess==True`).

In [14]:
stokes.solve(verbose=False, debug=False, zero_init_guess=True)

In [15]:
if 0 and uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(meshball)
    pvmesh.point_data["P"] = vis.scalar_fn_to_pv_points(pvmesh, p_soln.sym)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln)
    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)

    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        scalars="T",
        show_edges=True,
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=True,
    )

    pl.add_arrows(pvmesh_v.points, pvmesh_v.point_data["V"], mag=3e16 )


    # pl.show()

In [16]:
# Keep the initialisation separate
# so we can run the loop again in a notebook

max_steps = 25
timestep = 0
elapsed_time = uw.quantity(0, "Myr")

In [17]:
adv_diff.view(class_documentation=True)


# Advection-Diffusion Equation Solver (Scalar Variable)

This class provides a solver for the scalar Advection-Diffusion equation using the characteristics based Semi-Lagrange Crank-Nicholson method
which is described in Spiegelman & Katz, (2006).

$$
\color{Green}{\underbrace{ \Bigl[ \frac{\partial u}{\partial t} + \left( \mathbf{v} \cdot \nabla \right) u \Bigr]}_{\dot{\mathbf{u}}}} -
\nabla \cdot
        \color{Blue}{\underbrace{\Bigl[ \boldsymbol\kappa \nabla u \Bigr]}_{\mathbf{F}}} =
        \color{Maroon}{\underbrace{\Bigl[ f \Bigl] }_{\mathbf{h}}}
$$

The term $\mathbf{F}$ relates diffusive fluxes to gradients in the unknown $u$. The advective flux that results from having gradients along
the direction of transport (given by the velocity vector field $\mathbf{v}$ ) are included in the $\dot{\mathbf{u}}$ term.

The term $\dot{\mathbf{u}}$ involves upstream sampling to find the value $u^*$ which represents the value of $u$ at
the points which later arrive at the nodal points of the mesh. This is achieved using a "hidden"
swarm variable which is advected backwards from the nodal points automatically during the `solve` phase.

## Properties

  - The unknown is $u$.

  - The velocity field is $\mathbf{v}$ and is provided as a `sympy` function to allow operations such as time-averaging to be
    calculated in situ (e.g. `V_Field = v_solution.sym`) **NOTE: no it's not.

  - The diffusivity tensor, $\kappa$ is provided by setting the `constitutive_model` property to
    one of the scalar `uw.constitutive_models` classes and populating the parameters.
    It is usually a constant or a function of position / time and may also be non-linear
    or anisotropic.

  - Volumetric sources of $u$ are specified using the $f$ property and can be any valid combination of `sympy` functions of position and
    `meshVariable` or `swarmVariable` types.

## References

Spiegelman, M., & Katz, R. F. (2006). A semi-Lagrangian Crank-Nicolson algorithm for the numerical solution of advection-diffusion problems. Geochemistry, Geophysics, Geosystems, 7(4). https://doi.org/10.1029/2005GC001073



---

**Class**: <class 'underworld3.systems.solvers.SNES_AdvectionDiffusion'>

# Underworld / PETSc General Scalar Equation Solver

Primary problem: 

...

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Lower | $\left[\begin{matrix}0.283\end{matrix}\right]  $ | 
| **essential** | Upper | $\left[\begin{matrix}0.273\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [18]:
delta_t = adv_diff.estimate_dt()
delta_t.to("Myr")

<IPython.core.display.Latex object>

In [19]:
adv_diff.solve(timestep=delta_t , zero_init_guess=True)

In [23]:
# Null space ?

for step in range(0, max_steps):
    print(f"Timestep: {timestep}, dt: {delta_t.to("Myr")}, time: {elapsed_time}")
    
    stokes.solve(zero_init_guess=True)
    delta_t = 2 * adv_diff.estimate_dt()
    adv_diff.solve(timestep=delta_t, zero_init_guess=False, verbose=False)

    timestep += 1
    elapsed_time += delta_t



Timestep: 25, dt: 915.3215864738536 [megayear], time: 21746.117427870984 [megayear]
Timestep: 26, dt: 814.7408862135247 [megayear], time: 22560.858314084508 [megayear]
Timestep: 27, dt: 717.3767768447486 [megayear], time: 23278.235090929255 [megayear]
Timestep: 28, dt: 648.0160968149956 [megayear], time: 23926.25118774425 [megayear]
Timestep: 29, dt: 607.0475747561691 [megayear], time: 24533.298762500417 [megayear]
Timestep: 30, dt: 580.3582399466269 [megayear], time: 25113.657002447046 [megayear]
Timestep: 31, dt: 557.7588646404477 [megayear], time: 25671.415867087493 [megayear]
Timestep: 32, dt: 545.0901067799982 [megayear], time: 26216.50597386749 [megayear]
Timestep: 33, dt: 540.4817887561157 [megayear], time: 26756.987762623605 [megayear]
Timestep: 34, dt: 527.5316757257316 [megayear], time: 27284.519438349336 [megayear]
Timestep: 35, dt: 500.1975639023442 [megayear], time: 27784.71700225168 [megayear]
Timestep: 36, dt: 502.5883401474635 [megayear], time: 28287.305342399144 [megay

In [21]:
0/0

ZeroDivisionError: division by zero

In [22]:
# visualise it


if uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(meshball)
    pvmesh.point_data["P"] = vis.scalar_fn_to_pv_points(pvmesh, p_soln)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln)
    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)

    

    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        scalars="T",
        show_edges=True,
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=True,
    )

    pl.add_arrows(pvmesh_v.points, pvmesh_v.point_data["V"], mag=7e15 )


    pl.export_html("html5/annulus_convection_scaled.html")
    pl.show(cpos="xy", jupyter_backend="trame")

Widget(value='<iframe src="http://localhost:61525/index.html?ui=P_0x38187bec0_0&reconnect=auto" class="pyvista…

In [ ]:
#| fig-cap: "Interactive Image: Convection model output"
from IPython.display import IFrame

IFrame(src="html5/annulus_convection_scaled.html", width=500, height=400)

## Exercise - Null space

Based on our previous notebook, can you see how to calculate and (if necessary) remove rigid-body the rotation 
null-space from the solution ? 

The use of a coarse-level singular-value decomposition for the velocity solver should help, in this case, but sometimes
you can see that there is a rigid body rotation (look at the streamlines). It's wise to check and quantify the presence of 
the null space.

```python
    stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"
```

## Exercise - Heat flux

Could you calculate the radial heat flux field ? Its surface average value plotted against
time tells you if you have reached a steady state.

Hint:

$$
    Q_\textrm{surf} = \nabla T \cdot \hat{r} + T (\mathbf{v} \cdot \hat{r} )
$$ 

```python
    Q_surf = -meshball.vector.gradient(t_soln.sym).dot(unit_rvec) +\
                    t_soln.sym[0] * v_soln.sym.dot(unit_rvec)
```




### References

Schubert, G., Turcotte, D. L., & Olson, P. (2001). Mantle Convection in the Earth and Planets (1st ed.). Cambridge University Press. https://doi.org/10.1017/CBO9780511612879
